In [4]:
import csv

transformer_dict = {'No':0, 'Yes':1}
city_dict = {}
def getDataSet(file_path,size = 10000):
    y = []
    with open(file_path) as file_obj:
        reader = csv.reader(file_obj)
        x = list(reader)
    x.pop(0)
    x = x[-size:]
    x_temp = []
    for i in range(len(x)):
        if x[i][-1] != 'NA' and x[i][-2] !='NA':
            x_temp.append(x[i])

    x = x_temp
    for line in x:
        y.append(transformer_dict [line.pop(-1)])
        # 处理第一列 日期
        line[0] = ''.join(line[0].split('-'))
        # 处理城市
        location = line[1]
        city_dict.setdefault(location,len(city_dict)+1)
        line[1] = city_dict[location]
        # 删掉 5、6、7
        for i in range(3):
            line.pop(5)
        # 删掉9、10
        for i in range(2):
            line.pop(6)
        # 删掉倒数第四列
        line.pop(-4)
        line[-1] = transformer_dict[line[-1]]
        for i in range(len(line)):
            if line[i] == 'NA':
                line[i] = 0

    return x,y

In [5]:
filename='weatherAUS.csv'
x, y = getDataSet(filename,5000)

In [6]:
print(x[:10])
print(y[:10])

print((len(x)))
print(len(y))

[['20151024', 1, '5', '20.3', '0', '31', '2', '13', '74', '43', 0, 0, '8', '13.3', '19.1', 0], ['20151025', 1, '8.1', '20.4', '0', '31', '11', '19', '75', '55', 0, 0, '7', '13.1', '19.1', 0], ['20151026', 1, '10.9', '17.8', '0.6', '69', '33', '30', '75', '32', 0, 0, '8', '12.4', '16.9', 0], ['20151027', 1, '2.5', '19', '0', '31', '2', '19', '56', '36', 0, 0, 0, '10.1', '18', 0], ['20151028', 1, '3.9', '20.8', '0', '30', '2', '19', '71', '40', 0, 0, 0, '12.1', '20.1', 0], ['20151029', 1, '7.3', '21.7', '0', '31', '2', '17', '71', '40', 0, 0, '4', '13.5', '20.7', 0], ['20151030', 1, '7.4', '22.9', '0', '30', '6', '13', '65', '35', 0, 0, 0, '16.1', '21.6', 0], ['20151031', 1, '10', '20.3', '0', '31', '7', '11', '67', '68', 0, 0, '8', '16.7', '20', 0], ['20151101', 1, '14.1', '20.6', '0.4', '52', '6', '30', '91', '72', 0, 0, '8', '17.5', '18.1', 0], ['20151102', 1, '11.7', '17.6', '0', '48', '4', '22', '72', '47', 0, 0, '8', '16.2', '15.3', 0]]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
9870
9870


In [7]:
import numpy as np

def my_train_test_split(data:list, label:list, train_size_rate=0.7):
    length = len(data)
    separate_line = int(train_size_rate*length)

    data = np.array(data).astype(dtype='float32')
    label = np.array(label).astype(dtype='int64')

    x_train = data[:separate_line]
    x_test = data[separate_line:]
    y_train = label[:separate_line]
    y_test = label[separate_line:]
    return x_train, y_train, x_test,y_test



In [8]:
x_train, y_train, x_test,y_test = my_train_test_split(x,y)


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

pipeline = Pipeline([
    ('clf', SVC(kernel='rbf', gamma=0.01, C=100))
])
parameters = {
    'clf__gamma': (0.01, 0.03, 0.1, 0.3, 1),
    'clf__C': (0.1, 0.3, 1, 3, 10, 30),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=3, cv=2,verbose=1, scoring='accuracy')
grid_search.fit(x_train, y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(x_test)
print(classification_report(y_test, predictions))

Fitting 2 folds for each of 30 candidates, totalling 60 fits
Best score: 0.820
Best parameters set:
	clf__C: 0.1
	clf__gamma: 0.01
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      2603
           1       0.00      0.00      0.00       358

    accuracy                           0.88      2961
   macro avg       0.44      0.50      0.47      2961
weighted avg       0.77      0.88      0.82      2961



d:\python\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.metrics import accuracy_score
print('accuracy is : {}'.format(accuracy_score(y_test, predictions)))


accuracy is : 0.8790949003714961
